In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline  

import pandas as pd
import numpy as np

from datetime import datetime, timedelta
import parseIntervalFiles as pif
import parseActivityFiles as paf
#pun intended :)
import consolidateFiles as cf
import datacleaning as cl

## ! Parse activity files and parse interval files are to be replaced with the corresponding database queries as soon as they are available 

## 0 - Pipeline configuration 
* set the input/output directories, user id and verbose level

In [2]:
verbose = True
    
DATA_PATH = "C:\\Users\\ju\\GDrive\\Projects\\HeRV\\Data\\" 
# DATA_PATH = "/home/ju/GDrive/Projects/HeRV/Data/"
    
RAW_PATH = DATA_PATH + "Raw"
PRE_PATH = DATA_PATH + "PreProcessed"     

## 1 - Extract sessions from raw data (intervals + annotations files)

### 1.1 - List activities with session start/stop datetime
Parses activities files and prints all errors found (activities without start or stop, invalid dates, etc)

In [3]:
sessions = []
for user in range(7):
    print ("------- parsing activity files for user ", user, " -------")
    user_sess = paf.get_user_sessions(user, dirname=RAW_PATH)
    print ("------- parsing interval files for user ", user, " -------")
    user_sess = cf.sessions_add_beats(sessions=user_sess, dirname=RAW_PATH+"\\"+str(user), verbose=False)
    sessions.extend(user_sess)

------- parsing activity files for user  0  -------
reading act170929.csv ... 
reading act170930.csv ... 
reading act171001.csv ... 
reading act171003.csv ... 
reading act171004.csv ... 
reading act171005.csv ... 
orphan start in: ['2017-10-05 12:54:00', 'start', 'eat', 'sitting', '']
orphan start in: ['2017-10-05 13:48:39', 'start', 'rest-active', 'sitting', '']
orphan start in: ['2017-10-05 13:57:37', 'start', 'movement', 'standing', '']
orphan start in: ['2017-10-05 17:55:44', 'start', 'focused-active', 'sitting', '']
orphan start in: ['2017-10-05 21:53:45', 'start', 'rest-active', 'sitting', '']
orphan start in: ['2017-10-05 23:47:45', 'start', 'focused-active', 'sitting', 'sitting on bed, installing and using pandas']
reading act171006.csv ... 
reading act171010.csv ... 
reading act171011.csv ... 
reading act171012.csv ... 
orphan stop in: ['2017-10-12 14:09:16', 'stop', '', '']
reading act171013.csv ... 
reading act171014.csv ... 
orphan stop in: ['2017-10-14 2:14:21', 'stop', ''

In [4]:
df = pd.DataFrame(sessions)

### Include column with beats count for ease of use 

In [5]:
df['beatscount'] = df['rr'].apply(len)

In [6]:
df.sample(5)

,activity,duration,notes,posture,rr,start,stop,user,beatscount
308,focused-passive,600,NaN,sit,"[{'date': 2017-10-06 11:32:01, 'interval': 959...",2017-10-06 11:32:00,2017-10-06 11:42:00,2,799
137,focused-active,1380,NaN,sit,[],2017-11-05 22:00:00,2017-11-05 22:23:00,0,0
229,movement,2213,pet shop,stand,"[{'date': 2018-04-19 10:30:48, 'interval': 578...",2018-04-19 10:30:47,2018-04-19 11:07:40,0,4676
412,sleep,25,NaN,sit,"[{'date': 2017-11-02 06:24:17, 'interval': 996...",2017-11-02 06:24:15,2017-11-02 06:24:40,3,33
190,movement,994,NaN,stand,"[{'date': 2018-01-23 17:44:10, 'interval': 517...",2018-01-23 17:44:09,2018-01-23 18:00:43,0,1868


## 2 - Cleaning data

### 2.1 Removing outliers and artifacts

(Note: For now, beats won't be saved to the output file, so, when loading the sessions file and getting
beats from DB again, for example for fragmentation, they will have to be removed again. But this is a reasonably low-cost op)

In [7]:
df['rr'] = df['rr'].apply(cl.clean_rr_series)

In [8]:
df['beatscount_clean'] = df['rr'].apply(len)

In [9]:
df['removed_artifacts'] = df['beatscount'] - df['beatscount_clean']
df['beatscount'] = df['beatscount_clean']
df = df.drop(['beatscount_clean'], axis=1)

In [10]:
df.describe()

,duration,user,beatscount,removed_artifacts
count,476.000000,476.000000,476.000000,476.000000
mean,3291.945378,1.302521,3697.609244,21.340336
std,6996.048707,1.699806,6603.005196,60.203670
min,0.000000,0.000000,0.000000,0.000000
25%,840.000000,0.000000,934.500000,0.000000
50%,1320.000000,1.000000,1708.500000,1.000000
75%,2409.000000,2.000000,3583.750000,12.000000
max,80732.000000,6.000000,46856.000000,591.000000


### 2.2 Removing sessions without beats recorded or with too few of them

In [11]:
l = len(df)
df = df[df['beatscount'] > 100]
print(l - len(df), ' sessions out of', l, 'removed for lack of interval data')

44  sessions out of 476 removed for lack of interval data


### 2.3 Removing wrong activities and grouping the ones with too few examples

In [12]:
df.groupby('activity').count()['duration']

activity
commute               3
eat                  46
exercise-high        18
exercise-low          3
focused-active      103
focused-passive      44
household-chores     30
leisure-active        4
leisure-passive      45
movement             53
not-recorded          4
rest-active          27
rest-passive         18
sleep                34
Name: duration, dtype: int64

In [13]:
df[df['activity'] == 'moving']

,activity,duration,notes,posture,rr,start,stop,user,beatscount,removed_artifacts


In [14]:
# we have too few exercise sessions to classify intensity
df.at[df['activity']=='exercise-low', 'activity'] = 'exercise'
df.at[df['activity']=='exercise-high', 'activity'] = 'exercise'

# also we have too few leisure active sessions
df.at[df['activity']=='leisure-active', 'activity'] = 'leisure'
df.at[df['activity']=='leisure-passive', 'activity'] = 'leisure'

# these were just mispelled
df.at[df['activity']=='focused-passive ', 'activity'] = 'focused-passive'
df.at[df['activity']=='focused-active ', 'activity'] = 'focused-active'
df.at[df['activity']=='moving', 'activity'] = 'movement'

# and these are not categorized
df = df[df['activity']!='not-recorded']

In [15]:
df.groupby('activity').count()['duration']

activity
commute               3
eat                  46
exercise             21
focused-active      103
focused-passive      44
household-chores     30
leisure              49
movement             53
rest-active          27
rest-passive         18
sleep                34
Name: duration, dtype: int64

In [16]:
df.groupby('posture').count()['duration']

posture
lie            37
lying down     13
sit           223
sitting        53
stand          76
standing       26
Name: duration, dtype: int64

In [17]:
# correcting diverse spellings in postures
df.at[df.posture == 'lying down', 'posture'] = 'lie'
df.at[df.posture == 'sitting', 'posture'] = 'sit'
df.at[df.posture.isin(['standing', 'standing ']), 'posture'] = 'stand'

In [18]:
df.groupby('posture').count()['duration']

posture
lie       50
sit      276
stand    102
Name: duration, dtype: int64

## 3 - Aggregating data

In [19]:
dic = df.to_dict(orient='records')
for i in dic:
    i.update(cf.features_from_dic(i['rr']))
print(dic[0]['rmssd'])

239.316870462


In [20]:
df = pd.DataFrame(dic)
df.sample(3)

,activity,beatscount,duration,hf,hfnu,lf,lf_hf,lfnu,mhr,mrri,...,posture,removed_artifacts,rmssd,rr,sdnn,start,stop,total_power,user,vlf
120,focused-active,457,297,255.196503,28.888913,628.175260,2.461536,71.111087,90.993509,664.214442,...,sit,0,25.601492,"[{'date': 2017-11-07 17:28:36, 'interval': 689...",58.854880,2017-11-07 17:28:35,2017-11-07 17:33:32,1969.563488,0,1086.191724
79,focused-active,6348,4800,1528.723805,54.983944,1251.585649,0.818713,45.016056,79.682906,761.684152,...,sit,4,64.131928,"[{'date': 2017-10-15 03:10:01, 'interval': 655...",83.602937,2017-10-15 03:10:00,2017-10-15 04:30:00,4054.153933,0,1273.844480
381,leisure,1169,1155,343.204527,31.074829,761.240892,2.218039,68.925171,85.093970,709.351583,...,sit,1,30.094941,"[{'date': 2017-12-21 13:18:51, 'interval': 647...",53.759696,2017-12-21 13:18:50,2017-12-21 13:38:05,1622.403406,4,517.957987


In [21]:
df['sess_id'] = df.index
df_export = df.drop(['rr'], axis = 1)
df_export.sample(1)

,activity,beatscount,duration,hf,hfnu,lf,lf_hf,lfnu,mhr,mrri,...,posture,removed_artifacts,rmssd,sdnn,start,stop,total_power,user,vlf,sess_id
38,leisure,5221,4800,1185.521574,42.463387,1606.346088,1.35497,57.536613,82.823005,735.262593,...,sit,1,55.654472,90.666919,2017-10-10 21:20:00,2017-10-10 22:40:00,4267.657355,0,1475.789693,38


In [22]:
df_export.describe()

,beatscount,duration,hf,hfnu,lf,lf_hf,lfnu,mhr,mrri,nn50,pnn50,removed_artifacts,rmssd,sdnn,total_power,user,vlf,sess_id
count,428.000000,428.000000,428.000000,428.000000,428.000000,428.000000,428.000000,428.000000,428.000000,428.000000,428.000000,428.000000,428.000000,428.000000,428.000000,428.000000,428.000000,428.000000
mean,4081.591121,3267.848131,1989.477888,41.390057,1615.744004,1.975874,58.609943,88.411016,719.176531,955.056075,16.289243,23.630841,62.989840,87.652390,4847.699576,1.366822,1242.477684,213.500000
std,6839.613593,6157.953560,3766.783554,17.094828,1630.007122,1.580216,17.094828,19.061990,134.769246,2714.919093,15.198604,63.050214,50.573555,42.582895,6056.234128,1.732327,1043.768465,123.697211
min,105.000000,88.000000,14.709334,8.241489,36.995074,0.172079,14.681505,54.190501,368.977202,0.000000,0.000000,0.000000,5.402758,21.083352,150.076646,0.000000,52.162541,0.000000
25%,1142.500000,900.000000,363.270288,28.320942,631.648937,0.825150,45.209929,75.793296,634.700818,75.750000,5.213287,0.000000,31.785212,60.161017,1625.315347,0.000000,535.083961,106.750000
50%,1908.000000,1372.500000,645.988169,39.076515,1167.441982,1.559082,60.923485,83.645558,728.134056,229.000000,12.041788,1.000000,45.782560,76.370797,3037.753186,1.000000,972.482799,213.500000
75%,3948.500000,2452.500000,1363.820598,54.790071,1921.978133,2.530959,71.679058,96.573251,808.818529,500.000000,21.889793,14.000000,72.961189,100.802394,4737.724204,2.000000,1565.166522,320.250000
max,46856.000000,34980.000000,22414.537885,85.318495,10846.431290,11.133730,91.758511,164.397688,1144.581425,20895.000000,81.780538,591.000000,290.138931,253.787250,36726.950587,6.000000,6926.126266,427.000000


In [23]:
df_export.to_excel(PRE_PATH + "\\sessions.xlsx")